In [18]:
import sde
import networkx as nx
import random
import portion
import itertools as it

# General

In [10]:
def CPT(graph, values):
    nodes = list(graph.nodes())
    CPT = list(it.product(values, repeat=len(nodes)))
    return CPT

def possibleOutcomes(graph, values):
    nodes = list(graph.nodes())
    cpt = CPT(graph, values)
    
    list_outcomes = []
    for i in CPT:
        list_outcomes.append(dict(zip(nodes, i)))
    return possibleOutcomes

# Tessam

In [11]:
#updated to program expansion when a hidden node is a vacuous messsage
# Update2: returns a subgraph rather than list of nodes
# Update3: if a hidden note is a root node, a random edge is directed from a random observed node to the hidden node
def getSubset(dg, observed_node):
    """accepts a graph and node name, returns a subgraph of that node's neighbors and non-d-separated nodes
        (includes the original node also)"""
    #get all neighbors
    children = list(dg.successors(observed_node))
    parents = list(dg.predecessors(observed_node))
    
    #get all non-d-separated nodes
    non_d_separated = []
    for n in list(dg.nodes()):
        n_children = list(dg.predecessors(n))
        commonDescendants = list(set(children) & set(n_children))
        if commonDescendants:
            non_d_separated += commonDescendants
            
    node_subset = [observed_node] + children + parents + commonDescendants
    
    # check for vacuous nodes, add an edge/parent if exists
    hidden_nodes = getHiddenNodes(dg)
    subset_hidden_nodes = list(set(node_subset) & set(hidden_nodes))
    for h in subset_hidden_nodes:
        parents_shn = list(dg.predecessors(h))
        if not parents_shn: 
            obsNodes = getObservedNodes(dg)
            randParent = random.choice(obsNodes)
            dg.add_edge(randParent, h, weight= random.uniform(0,1))
            parents_shn = [randParent]
        if not list(set(parents_shn) & set(node_subset)):
            node_subset.append(parents_shn[0])
            
    subgraph = dg.subgraph(node_subset) #generate the subgraph
    return subgraph

In [12]:
#find all observed (weighted) nodes in a graph
def getObservedNodes(dg):
    """accepts a graph, returns a list of the observed nodes in the graph"""
    observed_nodes = []
    dict_values = nx.get_node_attributes(dg, 'value')
    
    for v in dict_values:
        if (dg.nodes[v]['value'] == True):
            observed_nodes.append(v)
            
    return observed_nodes

# find all unobserved nodes
def getUnobservedNodes(dg):
    """accepts graph, returns list of False nodes"""
    unobserved_nodes = []
    dict_values = nx.get_node_attributes(dg, 'value')
    
    for v in dict_values:
        if (dg.nodes[v]['value'] == False): # and (dg.nodes[v]['type'] == bool)
            unobserved_nodes.append(v)

Marginal probability interval

In [13]:
# pick a node, calculate the min marg prob
def margParents(dg, node):
    parents = list(dg.predecessors(node))
    
    observed_nodes = getObservedNodes(subdg)
    observed_evidence = list(set(parents) & set(observed_nodes))
    
    false_nodes = getUnobservedNodes(subdg)
    false_evidence = list(set(parents) & set(false_nodes))
    
    #calculate prob of node given parents
    if not parents:
        margprob = 0
    else:
        cp = []
        for p in parents:
            weight = dg[p][node]['weight']
            cp.append(weight)

        margprob = numpy.prod(cp)
    return margprob

In [14]:
def minMargInterval(dg, values, node):
    subdg = getSubset(dg, node)
    worlds = possibleOutcomes(subdg, values)
    
    sub_nodes = list(subdg.nodes())
    parents = list(subdg.predecessors(node))
    ancestors = list(set(nx.ancestors(subdg, node)) - set(parents))
    
    margprobs = []
    minMarg = []
    for w in worlds:
        
        for node in sub_nodes:
            node_value = w.get(node)
            subdg.nodes[node]['value'] = node_value
        
        marg1 = margParents(subdg, node)
        margprobs.append(marg1)

        marg2 = [margParents(dg, p) for p in parents]
        marg3 = [margParents(dg, a) for a in ancestors]
        margprobs + marg2 + marg3
        
        node_marg = numpy.prod(margprobs)
        minMarg.append(node_marg)
        
    #Lessam_min = sum(minMarg)
    return numpy.sum(minMarg)
            

In [15]:
# pick a node, calculate the min marg prob
def margChildren(dg, node):
    children = list(dg.successors(node))
    
    #calculate prob of children given node
    if not children:
        margprob = 0
    else:
        cp = []
        for c in children:
            weight = dg[node][c]['weight']
            cp.append(weight)

        margprob = numpy.prod(cp)
    return margprob

In [16]:
def maxMargInterval(dg, values, node):
    subdg = getSubset(dg, node)
    worlds = possibleOutcomes(subdg, values)
    
    sub_nodes = list(subdg.nodes())
    children = list(subdg.predecessors(node))
    descendants = list(set(nx.descendants(subdg, node)) - set(children))
    
    margprobs = []
    maxMarg = []
    for w in worlds:
        for node in sub_nodes:
            node_value = w.get(node)
            subdg.nodes[node]['value'] = node_value
            
        marg1 = margChildren(subdg, node)
        margprobs.append(marg1)

        marg2 = [margChildren(dg, c) for c in children]
        marg3 = [margChildren(dg, d) for d in descendants]
        margprobs + marg2 + marg3
        node_marg = numpy.prod(margprobs)
        
        maxMarg.append(node_marg)
        
    #Lessam_min = sum(minMarg)
    return numpy.sum(maxMarg)
            

In [17]:
# returns the interval-valued marginal probability of a hidden node 
def LessamInterval(dg, values, node):
    lower_bound = minMargInterval(dg, values, node)
    upper_bound = maxMargInterval(dg, values, node)
    marg_prob_interval = [lower_bound, upper_bound] #portion.closed(lower_bound, upper_bound)
    return marg_prob_interval
